# Projeto 1 - MLP
## SCC 5809 - Redes Neurais Artificiais e Aprendizado Profundo
**Nome**: <br>
**NUSP**: 

## Bibliotecas e implementação da MLP

In [ ]:
import os
import numpy as np
from numpy import random, exp

import pandas as pd
from math import ceil, log
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
# Função de ativação sigmoid
def sigmoid(x):
  return 1/(1 + np.exp(-x))

# Derivada da função sigmoid
def derivativeSigmoid(x):
  return x * (1 - x)

# Funcao de custo
def vectorDifference(out, y):
  return y - out

class MLP:

  def __init__(self, layers, seed=None):
    if seed is not None:
      np.random.seed(seed)

    # inicializados aleatoriamente de um distrib (media = 0 e desvio padrao = 1)
    self.weights = [random.randn(a, b) for a, b in zip(layers[:-1], layers[1:])]
    self.bias = [random.randn(1, a) for a in layers[1:]]

    # vetores para pesos anterior
    self.prev_weights = [np.zeros((a, b)) for a, b in zip(layers[:-1], layers[1:])]
    self.prev_bias = [np.zeros((1, a)) for a in layers[1:]]
    return

  # propaga a entrada na rede
  def forward(self, x, activationf=sigmoid):
    output_p_layer = [np.array(x)]
    for weight, bias in zip(self.weights, self.bias):
      x = activationf(np.dot(x, weight) + bias)
      output_p_layer.append(x)

    return x, output_p_layer

  # calculas os deltas para atualizacao dos pesos
  def calcDeltas(self, error, output_per_layer, dactivationf=derivativeSigmoid):
    deltas = [error * dactivationf(output_per_layer[-1])]
    for i in range(len(self.weights) - 1, 0, -1):
      delta_aux = deltas[len(deltas) - 1].dot(self.weights[i].T) * dactivationf(output_per_layer[i])
      deltas.append(delta_aux)
    deltas = deltas[::-1]

    return deltas

  # retropropaga o erro para as camadas, modificando pesos anteriores para otimizar a funcao de custo
  def backpropagation(self, x, y, lrate=0.5, alpha=0, activationf=sigmoid, dactivationf=derivativeSigmoid, costf=vectorDifference):
    output, output_per_layer = self.forward(x, activationf=activationf)
    error = costf(output, y)

    delta = self.calcDeltas(error, output_per_layer, dactivationf=dactivationf)

    for i, weight in enumerate(self.weights):
      if len(output_per_layer[i].shape) == 1:
        output_per_layer[i] = output_per_layer[i].reshape(1, len(output_per_layer[i]))
      grad_weight = (lrate * np.dot(output_per_layer[i].T, delta[i]))
      weight +=  grad_weight + (alpha * self.prev_weights[i])
      self.prev_weights[i] = grad_weight

    for i, bias in enumerate(self.bias):
      grad_bias = (lrate * delta[i])
      bias += grad_bias + (alpha * self.prev_bias[i])
      self.prev_bias[i] = grad_bias

    return

  def evaluate_accuracy(self, X, Y, threshold=0.5):
    n_correct = 0
    for sample, y in zip(X,Y):
      output, _ = self.forward(sample)
      thresholded_output = (output > threshold) * 1
      #print(f'output: {output}')
      #print(f'thresholded_output: {thresholded_output}')
      #print(f'y: {y}')
      if np.array_equal(thresholded_output[0], y):
      #if (thresholded_output[0] == y).all:
        n_correct += 1
    #print(f'n_correct = {n_correct}')
    #print(f"len(Y_data): {len(Y)}.")
    accuracy = n_correct/len(Y)
    #print(f"Accuracy = {accuracy}")

    return accuracy

  def train(self, X, Y, max_epochs=2000, lrate=0.5, alpha=0.5, tol=1e-3, costf=vectorDifference):
    error_list = []
    epoch = 0

    for epoch in range(max_epochs):
      epoch_error = []

      for sample, y in zip(X,Y):
        #print(sample)
        output, _ = self.forward(sample)
        epoch_error.append(costf(output, y))
        self.backpropagation(sample, y, lrate=lrate, alpha=alpha)

      mse = np.sum(np.square(epoch_error))/len(epoch_error)
      error_list.append(mse)

    accuracy = self.evaluate_accuracy(X,Y)
    return error_list, accuracy

  def test(self, X, Y, costf=vectorDifference):
    test_error = []
    for x,y in zip(X,Y):
      output, _ = self.forward(x)
      test_error.append(costf(output, y))
    MSE = np.sum(np.square(test_error)) / len(test_error)
    accuracy = self.evaluate_accuracy(X,Y)
    return MSE, accuracy

## Projeto

In [ ]:
result_csv_dir = './result_csv'
if not os.path.isdir(result_csv_dir):
   os.makedirs(result_csv_dir)

In [ ]:
def test_hiperparameters(X_data:pd.DataFrame, y_data:pd.DataFrame, classification=False, lrate:list=[0.1, 0.5, 0.9], alpha:list=[0, 0.2, 0.5, 0.7, 1], test_size_list:list=[0.2, 0.5, 0.8], n_intermed_layers:int=10, neur_intermed:int=10, max_epochs_list:list=[50,100,200], seed=312):

  #fig, axs = plt.subplots(len(lrate), len(alpha), sharex=True, sharey=True)
  #fig.set_figheight(2*len(lrate))
  #fig.set_figwidth(4*len(alpha))

  X_data, y_data = np.array(X_data), np.array(y_data)

  if classification:
    df = pd.DataFrame(columns=['#neurons por layer oculto', '#layers ocultos', 'max epochs', 'test size', 'lrate', 'alpha', 'accuracy(training)', 'accuracy(test)'])
  else:
    df= pd.DataFrame(columns=['#neurons por layer oculto', '#layers ocultos', 'max epochs', 'test size', 'lrate', 'alpha', 'MSE(training)', 'MSE(test)'])

  for test_size in test_size_list:
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=test_size, random_state=seed, shuffle=True)

    layers = [len(X_train[0])]
    layers += [neur_intermed for _ in range(n_intermed_layers)]
    layers += [len(y_train[0])]

    for i, lrate_i in enumerate(lrate):
      for j, alpha_i in enumerate(alpha):
        for k, max_epochs in enumerate(max_epochs_list):
          network = MLP(layers=layers, seed=seed)
          error, accuracy_train = network.train(X=X_train, Y=y_train, lrate=lrate_i, alpha=alpha_i, max_epochs=max_epochs)
          MSE, accuracy_test = network.test(X=X_test, Y=y_test)

          if classification:
            df = pd.concat([pd.DataFrame([[neur_intermed, n_intermed_layers, max_epochs, test_size, lrate_i, alpha_i, accuracy_train, accuracy_test]], columns=df.columns), df])
            print('max_epochs:{}\t test_size:{:.2f}\t  lrate:{:.2f}\t  alpha:{:.2f}\t  accuracy(training):{:.5f}\t  accuracy(test):{:.5f}'.format(max_epochs, test_size, lrate_i, alpha_i, accuracy_train, accuracy_test))
          else:
            df = pd.concat([pd.DataFrame([[neur_intermed, n_intermed_layers, max_epochs, test_size, lrate_i, alpha_i, error[-1], MSE]], columns=df.columns), df])
            print('max_epochs:{}\t test_size:{:.2f}\t  lrate:{:.2f}\t  alpha:{:.2f}\t  MSE(training):{:.5f}\t  MSE(test):{:.5f}'.format(max_epochs, test_size, lrate_i, alpha_i, error[-1], MSE))
          #axs[i,j].set_title('lrate:{:.2f}    alpha:{:.2f}'.format(lrate_i, alpha_i))
          #axs[i,j].plot(error)
          #axs[i,j].plot(len(error)-1, MSE, ls=None, marker='x', c='red')



  #plt.show()

  return df

# Wine - Classificação

## Pré-processamento de dados

In [ ]:
cols = ['class', 'alcohol', 'malic acid', 'ash', 'alkalinity of ash', 'magnesium', 'total phenols', 'flavanoids', 'nonflavanoid phenols', 'proanthocyanins', 'color intensity', 'hue', 'OD', 'proline']

# Carregando o dataset atraves do pandas
wine_df = pd.read_csv('data/wine.data', names=cols)
wine_df.head()

,class,alcohol,malic acid,ash,alkalinity of ash,magnesium,total phenols,flavanoids,nonflavanoid phenols,proanthocyanins,color intensity,hue,OD,proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [ ]:
# classificando a variavel resposta
wine_df['class'] = wine_df['class'].astype('category')
wine_df = pd.get_dummies(wine_df, columns=['class'])
wine_df.head()

,alcohol,malic acid,ash,alkalinity of ash,magnesium,total phenols,flavanoids,nonflavanoid phenols,proanthocyanins,color intensity,hue,OD,proline,class_1,class_2,class_3
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,True,False,False
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,True,False,False
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,True,False,False
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,True,False,False
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,True,False,False


In [ ]:
# Normalização
scaler = MinMaxScaler()
wine_norm = scaler.fit_transform(wine_df)
wine_norm = pd.DataFrame(wine_norm)
wine_norm.columns = wine_df.columns
wine_norm.head()

,alcohol,malic acid,ash,alkalinity of ash,magnesium,total phenols,flavanoids,nonflavanoid phenols,proanthocyanins,color intensity,hue,OD,proline,class_1,class_2,class_3
0,0.842105,0.191700,0.572193,0.257732,0.619565,0.627586,0.573840,0.283019,0.593060,0.372014,0.455285,0.970696,0.561341,1.0,0.0,0.0
1,0.571053,0.205534,0.417112,0.030928,0.326087,0.575862,0.510549,0.245283,0.274448,0.264505,0.463415,0.780220,0.550642,1.0,0.0,0.0
2,0.560526,0.320158,0.700535,0.412371,0.336957,0.627586,0.611814,0.320755,0.757098,0.375427,0.447154,0.695971,0.646933,1.0,0.0,0.0
3,0.878947,0.239130,0.609626,0.319588,0.467391,0.989655,0.664557,0.207547,0.558360,0.556314,0.308943,0.798535,0.857347,1.0,0.0,0.0
4,0.581579,0.365613,0.807487,0.536082,0.521739,0.627586,0.495781,0.490566,0.444795,0.259386,0.455285,0.608059,0.325963,1.0,0.0,0.0


In [ ]:
# Separando as features e labels
wine_X = wine_norm.drop(columns=['class_1','class_2','class_3'])
wine_y = wine_norm[['class_1','class_2','class_3']]
wine_X.head()

,alcohol,malic acid,ash,alkalinity of ash,magnesium,total phenols,flavanoids,nonflavanoid phenols,proanthocyanins,color intensity,hue,OD,proline
0,0.842105,0.191700,0.572193,0.257732,0.619565,0.627586,0.573840,0.283019,0.593060,0.372014,0.455285,0.970696,0.561341
1,0.571053,0.205534,0.417112,0.030928,0.326087,0.575862,0.510549,0.245283,0.274448,0.264505,0.463415,0.780220,0.550642
2,0.560526,0.320158,0.700535,0.412371,0.336957,0.627586,0.611814,0.320755,0.757098,0.375427,0.447154,0.695971,0.646933
3,0.878947,0.239130,0.609626,0.319588,0.467391,0.989655,0.664557,0.207547,0.558360,0.556314,0.308943,0.798535,0.857347
4,0.581579,0.365613,0.807487,0.536082,0.521739,0.627586,0.495781,0.490566,0.444795,0.259386,0.455285,0.608059,0.325963


In [ ]:
wine_y.head()

,class_1,class_2,class_3
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0


## Aplicação

### Testes com 1 camada oculta

In [ ]:
df_1InNe_1HidLa_wine = test_hiperparameters(X_data = wine_X, y_data = wine_y, classification=True, n_intermed_layers = 1, neur_intermed = 1)

<ipython-input-3-b5beb518ad31>:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([pd.DataFrame([[neur_intermed, n_intermed_layers, max_epochs, test_size, lrate_i, alpha_i, accuracy_train, accuracy_test]], columns=df.columns), df])


max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.00	  accuracy(training):0.60563	  accuracy(test):0.77778
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.00	  accuracy(training):0.65493	  accuracy(test):0.80556
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.00	  accuracy(training):0.69014	  accuracy(test):0.75000
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.20	  accuracy(training):0.61972	  accuracy(test):0.80556
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.20	  accuracy(training):0.66901	  accuracy(test):0.77778
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.20	  accuracy(training):0.70423	  accuracy(test):0.75000
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.50	  accuracy(training):0.64085	  accuracy(test):0.80556
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.50	  accuracy(training):0.69014	  accuracy(test):0.75000
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.50	  accuracy(training):0.68310	  accuracy(test):0.72222
max_

In [ ]:
df_1InNe_1HidLa_wine.head()
# df_1InNe_1HidLa_wine.loc[df_1InNe_1HidLa_wine['test size'] == 0.8]
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None
                       ):
  display(df_1InNe_1HidLa_wine.sort_values(by=['accuracy(test)'], ascending=False))
df_1InNe_1HidLa_wine.to_csv(os.path.join(result_csv_dir, 'df_1InNe_1HidLa_wine.csv'), index=False)

,#neurons por layer oculto,#layers ocultos,max epochs,test size,lrate,alpha,accuracy(training),accuracy(test)
0,1,1,50,0.2,0.1,0.7,0.647887,0.805556
0,1,1,100,0.2,0.1,0,0.654930,0.805556
0,1,1,50,0.2,0.1,0.2,0.619718,0.805556
0,1,1,50,0.2,0.1,1,0.661972,0.805556
0,1,1,50,0.2,0.1,0.5,0.640845,0.805556
0,1,1,50,0.2,0.1,0,0.605634,0.777778
0,1,1,100,0.2,0.1,0.2,0.669014,0.777778
0,1,1,200,0.2,0.9,1,0.661972,0.750000
0,1,1,100,0.2,0.1,0.7,0.690141,0.750000
0,1,1,100,0.2,0.1,1,0.697183,0.750000


In [ ]:
df_10InNe_1HidLa_wine = test_hiperparameters(X_data = wine_X, y_data = wine_y, classification=True, n_intermed_layers = 1, neur_intermed = 10)

<ipython-input-3-b5beb518ad31>:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([pd.DataFrame([[neur_intermed, n_intermed_layers, max_epochs, test_size, lrate_i, alpha_i, accuracy_train, accuracy_test]], columns=df.columns), df])


max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.00	  accuracy(training):0.97887	  accuracy(test):1.00000
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.00	  accuracy(training):0.98592	  accuracy(test):1.00000
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.00	  accuracy(training):0.99296	  accuracy(test):1.00000
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.20	  accuracy(training):0.98592	  accuracy(test):1.00000
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.20	  accuracy(training):0.99296	  accuracy(test):1.00000
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.20	  accuracy(training):0.99296	  accuracy(test):1.00000
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.50	  accuracy(training):0.98592	  accuracy(test):1.00000
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.50	  accuracy(training):0.99296	  accuracy(test):1.00000
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.50	  accuracy(training):1.00000	  accuracy(test):1.00000
max_

In [ ]:
df_10InNe_1HidLa_wine.head()
# df_10InNe_1HidLa_wine.loc[df_10InNe_1HidLa_wine['test size'] == 0.8]
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None
                       ):
  display(df_10InNe_1HidLa_wine.sort_values(by=['accuracy(test)'], ascending=False))
df_10InNe_1HidLa_wine.to_csv(os.path.join(result_csv_dir, 'df_10InNe_1HidLa_wine.csv'), index=False)

,#neurons por layer oculto,#layers ocultos,max epochs,test size,lrate,alpha,accuracy(training),accuracy(test)
0,10,1,50,0.2,0.1,0,0.978873,1.000000
0,10,1,200,0.2,0.1,0.5,1.000000,1.000000
0,10,1,50,0.2,0.5,0,0.992958,1.000000
0,10,1,200,0.2,0.1,1,1.000000,1.000000
0,10,1,100,0.2,0.1,1,0.992958,1.000000
0,10,1,50,0.2,0.1,1,0.992958,1.000000
0,10,1,100,0.2,0.1,0.7,0.992958,1.000000
0,10,1,50,0.2,0.1,0.7,0.985915,1.000000
0,10,1,200,0.2,0.1,0.7,1.000000,1.000000
0,10,1,100,0.2,0.1,0.5,0.992958,1.000000


In [ ]:
df_100InNe_1HidLa_wine = test_hiperparameters(X_data = wine_X, y_data = wine_y, classification=True, n_intermed_layers = 1, neur_intermed = 100)

<ipython-input-3-b5beb518ad31>:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([pd.DataFrame([[neur_intermed, n_intermed_layers, max_epochs, test_size, lrate_i, alpha_i, accuracy_train, accuracy_test]], columns=df.columns), df])


max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.00	  accuracy(training):0.98592	  accuracy(test):0.97222
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.00	  accuracy(training):1.00000	  accuracy(test):0.97222
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.00	  accuracy(training):1.00000	  accuracy(test):0.97222
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.20	  accuracy(training):0.99296	  accuracy(test):0.97222
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.20	  accuracy(training):1.00000	  accuracy(test):0.97222
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.20	  accuracy(training):1.00000	  accuracy(test):0.97222
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.50	  accuracy(training):0.99296	  accuracy(test):0.97222
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.50	  accuracy(training):1.00000	  accuracy(test):0.97222
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.50	  accuracy(training):1.00000	  accuracy(test):0.97222
max_

In [ ]:
df_100InNe_1HidLa_wine.head()
# df_100InNe_1HidLa_wine.loc[df_100InNe_1HidLa_wine['test size'] == 0.8]
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None
                       ):
  display(df_100InNe_1HidLa_wine.sort_values(by=['accuracy(test)'], ascending=False))
df_100InNe_1HidLa_wine.to_csv(os.path.join(result_csv_dir, 'df_100InNe_1HidLa_wine.csv'), index=False)

,#neurons por layer oculto,#layers ocultos,max epochs,test size,lrate,alpha,accuracy(training),accuracy(test)
0,100,1,50,0.2,0.1,0,0.985915,0.972222
0,100,1,200,0.2,0.1,0.5,1.000000,0.972222
0,100,1,100,0.2,0.9,1,1.000000,0.972222
0,100,1,50,0.2,0.9,1,1.000000,0.972222
0,100,1,100,0.2,0.1,1,1.000000,0.972222
0,100,1,50,0.2,0.1,1,0.992958,0.972222
0,100,1,200,0.2,0.1,0.7,1.000000,0.972222
0,100,1,100,0.2,0.1,0.7,1.000000,0.972222
0,100,1,50,0.2,0.1,0.7,0.992958,0.972222
0,100,1,200,0.2,0.9,1,1.000000,0.972222


### Testes com 2 camadas ocultas

In [ ]:
df_1InNe_2HidLa_wine = test_hiperparameters(X_data = wine_X, y_data = wine_y, classification=True, n_intermed_layers = 2, neur_intermed = 1)

<ipython-input-3-b5beb518ad31>:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([pd.DataFrame([[neur_intermed, n_intermed_layers, max_epochs, test_size, lrate_i, alpha_i, accuracy_train, accuracy_test]], columns=df.columns), df])


max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.00	  accuracy(training):0.63380	  accuracy(test):0.41667
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.00	  accuracy(training):0.64789	  accuracy(test):0.41667
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.00	  accuracy(training):0.64789	  accuracy(test):0.41667
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.20	  accuracy(training):0.64789	  accuracy(test):0.41667
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.20	  accuracy(training):0.64789	  accuracy(test):0.41667
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.20	  accuracy(training):0.64789	  accuracy(test):0.41667
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.50	  accuracy(training):0.64789	  accuracy(test):0.41667
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.50	  accuracy(training):0.64789	  accuracy(test):0.41667
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.50	  accuracy(training):0.64789	  accuracy(test):0.41667
max_

In [ ]:
df_1InNe_2HidLa_wine.head()
# df_1InNe_2HidLa_wine.loc[df_1InNe_2HidLa_wine['test size'] == 0.8]
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None
                       ):
  display(df_1InNe_2HidLa_wine.sort_values(by=['accuracy(test)'], ascending=False))
df_1InNe_2HidLa_wine.to_csv(os.path.join(result_csv_dir, 'df_1InNe_2HidLa_wine.csv'), index=False)

,#neurons por layer oculto,#layers ocultos,max epochs,test size,lrate,alpha,accuracy(training),accuracy(test)
0,1,2,200,0.8,0.9,1,0.714286,0.573427
0,1,2,100,0.8,0.9,1,0.714286,0.573427
0,1,2,50,0.8,0.9,1,0.714286,0.573427
0,1,2,200,0.8,0.9,0.7,0.714286,0.573427
0,1,2,100,0.8,0.9,0.7,0.714286,0.573427
0,1,2,50,0.8,0.9,0.7,0.714286,0.573427
0,1,2,200,0.8,0.9,0.5,0.714286,0.573427
0,1,2,50,0.8,0.9,0.5,0.714286,0.573427
0,1,2,100,0.8,0.9,0.5,0.714286,0.566434
0,1,2,200,0.8,0.1,0.2,0.714286,0.566434


In [ ]:
df_10InNe_2HidLa_wine = test_hiperparameters(X_data = wine_X, y_data = wine_y, classification=True, n_intermed_layers = 2, neur_intermed = 10)

<ipython-input-3-b5beb518ad31>:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([pd.DataFrame([[neur_intermed, n_intermed_layers, max_epochs, test_size, lrate_i, alpha_i, accuracy_train, accuracy_test]], columns=df.columns), df])


max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.00	  accuracy(training):0.98592	  accuracy(test):0.97222
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.00	  accuracy(training):0.99296	  accuracy(test):0.94444
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.00	  accuracy(training):1.00000	  accuracy(test):0.94444
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.20	  accuracy(training):0.98592	  accuracy(test):0.97222
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.20	  accuracy(training):0.99296	  accuracy(test):0.94444
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.20	  accuracy(training):1.00000	  accuracy(test):0.94444
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.50	  accuracy(training):0.98592	  accuracy(test):0.97222
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.50	  accuracy(training):0.99296	  accuracy(test):0.94444
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.50	  accuracy(training):1.00000	  accuracy(test):0.94444
max_

In [ ]:
df_10InNe_2HidLa_wine.head()
# df_10InNe_2HidLa_wine.loc[df_10InNe_2HidLa_wine['test size'] == 0.8]
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None
                       ):
  display(df_10InNe_2HidLa_wine.sort_values(by=['accuracy(test)'], ascending=False))
df_10InNe_2HidLa_wine.to_csv(os.path.join(result_csv_dir, 'df_10InNe_2HidLa_wine.csv'), index=False)

,#neurons por layer oculto,#layers ocultos,max epochs,test size,lrate,alpha,accuracy(training),accuracy(test)
0,10,2,50,0.2,0.1,0,0.985915,0.972222
0,10,2,50,0.2,0.1,0.2,0.985915,0.972222
0,10,2,50,0.2,0.1,0.5,0.985915,0.972222
0,10,2,50,0.2,0.1,0.7,0.985915,0.972222
0,10,2,100,0.5,0.1,0,1.000000,0.966292
0,10,2,200,0.5,0.5,0.5,1.000000,0.955056
0,10,2,50,0.5,0.9,1,1.000000,0.955056
0,10,2,200,0.5,0.9,0.7,1.000000,0.955056
0,10,2,100,0.5,0.9,0.7,1.000000,0.955056
0,10,2,50,0.5,0.9,0.7,1.000000,0.955056


In [ ]:
df_100InNe_2HidLa_wine = test_hiperparameters(X_data = wine_X, y_data = wine_y, classification=True, n_intermed_layers = 2, neur_intermed = 100)

<ipython-input-3-b5beb518ad31>:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([pd.DataFrame([[neur_intermed, n_intermed_layers, max_epochs, test_size, lrate_i, alpha_i, accuracy_train, accuracy_test]], columns=df.columns), df])


max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.00	  accuracy(training):1.00000	  accuracy(test):1.00000
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.00	  accuracy(training):1.00000	  accuracy(test):1.00000
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.00	  accuracy(training):1.00000	  accuracy(test):0.97222
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.20	  accuracy(training):1.00000	  accuracy(test):1.00000
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.20	  accuracy(training):1.00000	  accuracy(test):0.97222
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.20	  accuracy(training):1.00000	  accuracy(test):0.97222
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.50	  accuracy(training):1.00000	  accuracy(test):1.00000
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.50	  accuracy(training):1.00000	  accuracy(test):0.97222
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.50	  accuracy(training):1.00000	  accuracy(test):0.97222
max_

In [ ]:
df_100InNe_2HidLa_wine.head()
# df_100InNe_2HidLa_wine.loc[df_100InNe_2HidLa_wine['test size'] == 0.8]
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None
                       ):
  display(df_100InNe_2HidLa_wine.sort_values(by=['accuracy(test)'], ascending=False))
df_100InNe_2HidLa_wine.to_csv(os.path.join(result_csv_dir, 'df_100InNe_2HidLa_wine.csv'), index=False)

,#neurons por layer oculto,#layers ocultos,max epochs,test size,lrate,alpha,accuracy(training),accuracy(test)
0,100,2,50,0.2,0.1,0,1.000000,1.000000
0,100,2,100,0.2,0.1,0,1.000000,1.000000
0,100,2,50,0.2,0.1,0.2,1.000000,1.000000
0,100,2,50,0.2,0.1,0.5,1.000000,1.000000
0,100,2,50,0.2,0.1,0.7,1.000000,1.000000
0,100,2,50,0.2,0.5,0.2,1.000000,1.000000
0,100,2,100,0.2,0.9,0.7,0.873239,1.000000
0,100,2,100,0.5,0.1,0.7,1.000000,0.988764
0,100,2,100,0.5,0.5,0,1.000000,0.988764
0,100,2,50,0.5,0.5,0,1.000000,0.988764


# Música - Regressão

## Pré-processamento de dados

In [ ]:
# Carregando o dataset
music_df = pd.read_csv('data/default_features_1059_tracks.txt', header=None)
music_df.head()

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,7.161286,7.835325,2.911583,0.984049,-1.499546,-2.094097,0.576000,-1.205671,1.849122,-0.425598,...,-1.504263,0.351267,-1.018726,-0.174878,-1.089543,-0.668840,-0.914772,-0.836250,-15.75,-47.95
1,0.225763,-0.094169,-0.603646,0.497745,0.874036,0.290280,-0.077659,-0.887385,0.432062,-0.093963,...,-0.495712,-0.465077,-0.157861,-0.157189,0.380951,1.088478,-0.123595,1.391141,14.91,-23.51
2,-0.692525,-0.517801,-0.788035,1.214351,-0.907214,0.880213,0.406899,-0.694895,-0.901869,-1.701574,...,-0.637167,0.147260,0.217914,2.718442,0.972919,2.081069,1.375763,1.063847,12.65,-8.00
3,-0.735562,-0.684055,2.058215,0.716328,-0.011393,0.805396,1.497982,0.114752,0.692847,0.052377,...,-0.178325,-0.065059,-0.724247,-1.020687,-0.751380,-0.385005,-0.012326,-0.392197,9.03,38.74
4,0.570272,0.273157,-0.279214,0.083456,1.049331,-0.869295,-0.265858,-0.401676,-0.872639,1.147483,...,-0.919463,-0.667912,-0.820172,-0.190488,0.306974,0.119658,0.271838,1.289783,34.03,-6.85


In [ ]:
# Normalizacao
music_scaler = MinMaxScaler()
music_norm = music_scaler.fit_transform(music_df)
music_norm = pd.DataFrame(music_norm)
music_norm.head()

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,0.792304,0.763528,0.554789,0.459774,0.332975,0.312870,0.694223,0.485061,0.772537,0.621409,...,0.010367,0.279176,0.059900,0.180836,0.046037,0.075438,0.046425,0.067519,0.217271,0.171557
1,0.160018,0.113349,0.072690,0.406624,0.751846,0.641129,0.601004,0.522885,0.619012,0.653455,...,0.122493,0.161148,0.146472,0.183240,0.260799,0.304061,0.166138,0.372358,0.558013,0.274298
2,0.076301,0.078613,0.047402,0.484945,0.437505,0.722346,0.670107,0.545760,0.474494,0.498113,...,0.106767,0.249681,0.184261,0.573916,0.347254,0.433194,0.393006,0.327565,0.532896,0.339499
3,0.072377,0.064981,0.437753,0.430514,0.595593,0.712046,0.825708,0.641975,0.647266,0.667595,...,0.157779,0.218983,0.089514,0.065927,0.095425,0.112365,0.182974,0.128292,0.492665,0.535985
4,0.191425,0.143468,0.117185,0.361344,0.782781,0.481490,0.574164,0.580605,0.477661,0.773414,...,0.075383,0.131821,0.079867,0.178716,0.249995,0.178020,0.225971,0.358486,0.770505,0.344333


In [ ]:
# Separando as features e labels
music_X = music_norm.drop(columns=[68,69])
music_y = music_norm[[68,69]]
music_X.head()

,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,64,65,66,67
0,0.792304,0.763528,0.554789,0.459774,0.332975,0.312870,0.694223,0.485061,0.772537,0.621409,...,0.080394,0.252689,0.010367,0.279176,0.059900,0.180836,0.046037,0.075438,0.046425,0.067519
1,0.160018,0.113349,0.072690,0.406624,0.751846,0.641129,0.601004,0.522885,0.619012,0.653455,...,0.213051,0.111763,0.122493,0.161148,0.146472,0.183240,0.260799,0.304061,0.166138,0.372358
2,0.076301,0.078613,0.047402,0.484945,0.437505,0.722346,0.670107,0.545760,0.474494,0.498113,...,0.102961,0.172823,0.106767,0.249681,0.184261,0.573916,0.347254,0.433194,0.393006,0.327565
3,0.072377,0.064981,0.437753,0.430514,0.595593,0.712046,0.825708,0.641975,0.647266,0.667595,...,0.101781,0.285450,0.157779,0.218983,0.089514,0.065927,0.095425,0.112365,0.182974,0.128292
4,0.191425,0.143468,0.117185,0.361344,0.782781,0.481490,0.574164,0.580605,0.477661,0.773414,...,0.198040,0.181444,0.075383,0.131821,0.079867,0.178716,0.249995,0.178020,0.225971,0.358486


In [ ]:
music_y.head()

,68,69
0,0.217271,0.171557
1,0.558013,0.274298
2,0.532896,0.339499
3,0.492665,0.535985
4,0.770505,0.344333


## Aplicação

### 1 camada intermediaria

In [ ]:
df_1InNe_1HidLa_music = test_hiperparameters(X_data = music_X, y_data = music_y, classification=False, n_intermed_layers = 1, neur_intermed = 1)

<ipython-input-3-b5beb518ad31>:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([pd.DataFrame([[neur_intermed, n_intermed_layers, max_epochs, test_size, lrate_i, alpha_i, error[-1], MSE]], columns=df.columns), df])


max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.00	  MSE(training):0.07600	  MSE(test):0.09369
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.00	  MSE(training):0.07396	  MSE(test):0.09101
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.00	  MSE(training):0.07049	  MSE(test):0.08937
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.20	  MSE(training):0.07567	  MSE(test):0.09321
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.20	  MSE(training):0.07328	  MSE(test):0.09066
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.20	  MSE(training):0.06981	  MSE(test):0.08924
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.50	  MSE(training):0.07523	  MSE(test):0.09268
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.50	  MSE(training):0.07232	  MSE(test):0.09042
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.50	  MSE(training):0.06913	  MSE(test):0.08913
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.70	  MSE(training):0.07496	  MSE(test):0.

In [ ]:
df_1InNe_1HidLa_music.head()
# df_1InNe_1HidLa_music.loc[df_1InNe_1HidLa_music['test size'] == 0.8]
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None
                       ):
  display(df_1InNe_1HidLa_music.sort_values(by=['MSE(test)']))
df_1InNe_1HidLa_music.to_csv(os.path.join(result_csv_dir, 'df_1InNe_1HidLa_music.csv'), index=False)

,#neurons por layer oculto,#layers ocultos,max epochs,test size,lrate,alpha,MSE(training),MSE(test)
0,1,1,200,0.5,0.1,0.5,0.068402,0.081279
0,1,1,200,0.5,0.1,0.7,0.067903,0.081289
0,1,1,200,0.5,0.1,0.2,0.069391,0.081312
0,1,1,200,0.5,0.1,1,0.067339,0.081345
0,1,1,200,0.5,0.1,0,0.070257,0.081410
0,1,1,100,0.5,0.1,1,0.070894,0.081978
0,1,1,100,0.5,0.1,0.7,0.071557,0.082118
0,1,1,100,0.5,0.1,0.5,0.072037,0.082259
0,1,1,100,0.5,0.5,0,0.068107,0.082360
0,1,1,100,0.5,0.5,0.2,0.067984,0.082395


In [ ]:
df_10InNe_1HidLa_music = test_hiperparameters(X_data = music_X, y_data = music_y, classification=False, n_intermed_layers = 1, neur_intermed = 10)

<ipython-input-3-b5beb518ad31>:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([pd.DataFrame([[neur_intermed, n_intermed_layers, max_epochs, test_size, lrate_i, alpha_i, error[-1], MSE]], columns=df.columns), df])


max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.00	  MSE(training):0.06968	  MSE(test):0.08690
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.00	  MSE(training):0.06544	  MSE(test):0.08447
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.00	  MSE(training):0.05946	  MSE(test):0.08492
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.20	  MSE(training):0.06894	  MSE(test):0.08625
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.20	  MSE(training):0.06441	  MSE(test):0.08470
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.20	  MSE(training):0.05775	  MSE(test):0.08476
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.50	  MSE(training):0.06803	  MSE(test):0.08572
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.50	  MSE(training):0.06296	  MSE(test):0.08504
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.50	  MSE(training):0.05565	  MSE(test):0.08379
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.70	  MSE(training):0.06753	  MSE(test):0.

In [ ]:
df_10InNe_1HidLa_music.head()
# df_10InNe_1HidLa_music.loc[df_10InNe_1HidLa_music['test size'] == 0.8]
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None
                       ):
  display(df_10InNe_1HidLa_music.sort_values(by=['MSE(test)']))
df_10InNe_1HidLa_music.to_csv(os.path.join(result_csv_dir, 'df_10InNe_1HidLa_music.csv'), index=False)

,#neurons por layer oculto,#layers ocultos,max epochs,test size,lrate,alpha,MSE(training),MSE(test)
0,10,1,200,0.5,0.1,0.5,0.055987,0.077604
0,10,1,200,0.5,0.1,0.7,0.055028,0.077621
0,10,1,200,0.5,0.1,0.2,0.057548,0.077910
0,10,1,200,0.5,0.1,1,0.053469,0.077913
0,10,1,200,0.5,0.1,0,0.058885,0.078185
0,10,1,100,0.5,0.1,1,0.060679,0.078374
0,10,1,50,0.5,0.5,0,0.060386,0.078390
0,10,1,50,0.5,0.5,0.2,0.059199,0.078568
0,10,1,100,0.5,0.1,0.7,0.061768,0.078786
0,10,1,50,0.5,0.9,0,0.056908,0.078944


In [ ]:
df_100InNe_1HidLa_music = test_hiperparameters(X_data = music_X, y_data = music_y, classification=False, n_intermed_layers = 1, neur_intermed = 100)

<ipython-input-3-b5beb518ad31>:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([pd.DataFrame([[neur_intermed, n_intermed_layers, max_epochs, test_size, lrate_i, alpha_i, error[-1], MSE]], columns=df.columns), df])


max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.00	  MSE(training):0.06096	  MSE(test):0.08324
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.00	  MSE(training):0.05315	  MSE(test):0.07957
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.00	  MSE(training):0.04360	  MSE(test):0.07868
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.20	  MSE(training):0.06040	  MSE(test):0.08193
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.20	  MSE(training):0.05197	  MSE(test):0.07960
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.20	  MSE(training):0.04182	  MSE(test):0.07969
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.50	  MSE(training):0.05978	  MSE(test):0.08123
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.50	  MSE(training):0.05062	  MSE(test):0.08056
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.50	  MSE(training):0.03950	  MSE(test):0.08161
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.70	  MSE(training):0.05944	  MSE(test):0.

In [ ]:
df_100InNe_1HidLa_music.head()
# df_100InNe_1HidLa_music.loc[df_100InNe_1HidLa_music['test size'] == 0.8]
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None
                       ):
  display(df_100InNe_1HidLa_music.sort_values(by=['MSE(test)']))
df_100InNe_1HidLa_music.to_csv(os.path.join(result_csv_dir, 'df_100InNe_1HidLa_music.csv'), index=False)

,#neurons por layer oculto,#layers ocultos,max epochs,test size,lrate,alpha,MSE(training),MSE(test)
0,100,1,200,0.2,0.1,0,0.043603,0.078677
0,100,1,100,0.2,0.1,0,0.053151,0.079569
0,100,1,100,0.2,0.1,0.2,0.051968,0.079599
0,100,1,200,0.2,0.1,0.2,0.041821,0.079685
0,100,1,100,0.2,0.9,1,0.030285,0.079741
0,100,1,100,0.2,0.9,0.5,0.026759,0.079999
0,100,1,100,0.2,0.1,0.5,0.050619,0.080561
0,100,1,50,0.2,0.9,1,0.052928,0.080651
0,100,1,50,0.2,0.9,0.7,0.039729,0.080652
0,100,1,50,0.2,0.9,0.5,0.042468,0.080927


### 2 camadas intermediarias

In [ ]:
df_1InNe_2HidLa_music = test_hiperparameters(X_data = music_X, y_data = music_y, classification=False, n_intermed_layers = 2, neur_intermed = 1)

<ipython-input-3-b5beb518ad31>:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([pd.DataFrame([[neur_intermed, n_intermed_layers, max_epochs, test_size, lrate_i, alpha_i, error[-1], MSE]], columns=df.columns), df])


max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.00	  MSE(training):0.07663	  MSE(test):0.09476
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.00	  MSE(training):0.07459	  MSE(test):0.09199
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.00	  MSE(training):0.07097	  MSE(test):0.09034
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.20	  MSE(training):0.07625	  MSE(test):0.09426
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.20	  MSE(training):0.07401	  MSE(test):0.09163
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.20	  MSE(training):0.07003	  MSE(test):0.09022
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.50	  MSE(training):0.07579	  MSE(test):0.09373
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.50	  MSE(training):0.07316	  MSE(test):0.09142
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.50	  MSE(training):0.06909	  MSE(test):0.09002
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.70	  MSE(training):0.07553	  MSE(test):0.

In [ ]:
df_1InNe_2HidLa_music.head()
# df_1InNe_2HidLa_music.loc[df_1InNe_2HidLa_music['test size'] == 0.8]
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None
                       ):
  display(df_1InNe_2HidLa_music.sort_values(by=['MSE(test)']))
df_1InNe_2HidLa_music.to_csv(os.path.join(result_csv_dir, 'df_1InNe_2HidLa_music.csv'), index=False)

,#neurons por layer oculto,#layers ocultos,max epochs,test size,lrate,alpha,MSE(training),MSE(test)
0,1,2,200,0.5,0.1,1,0.067510,0.081383
0,1,2,200,0.5,0.1,0.7,0.068341,0.081406
0,1,2,200,0.5,0.1,0.5,0.069055,0.081458
0,1,2,200,0.5,0.1,0.2,0.070451,0.081656
0,1,2,200,0.5,0.1,0,0.071455,0.081864
0,1,2,100,0.5,0.5,0,0.068515,0.082207
0,1,2,100,0.5,0.5,0.2,0.068317,0.082225
0,1,2,100,0.5,0.5,0.5,0.068437,0.082400
0,1,2,100,0.5,0.5,0.7,0.068672,0.082546
0,1,2,100,0.5,0.1,1,0.072042,0.082559


In [ ]:
df_10InNe_2HidLa_music = test_hiperparameters(X_data = music_X, y_data = music_y, classification=False, n_intermed_layers = 2, neur_intermed = 10)

<ipython-input-3-b5beb518ad31>:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([pd.DataFrame([[neur_intermed, n_intermed_layers, max_epochs, test_size, lrate_i, alpha_i, error[-1], MSE]], columns=df.columns), df])


max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.00	  MSE(training):0.07157	  MSE(test):0.08830
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.00	  MSE(training):0.06603	  MSE(test):0.08405
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.00	  MSE(training):0.05806	  MSE(test):0.07967
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.20	  MSE(training):0.07060	  MSE(test):0.08735
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.20	  MSE(training):0.06464	  MSE(test):0.08332
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.20	  MSE(training):0.05603	  MSE(test):0.07937
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.50	  MSE(training):0.06931	  MSE(test):0.08624
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.50	  MSE(training):0.06283	  MSE(test):0.08234
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.50	  MSE(training):0.05374	  MSE(test):0.07988
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.70	  MSE(training):0.06855	  MSE(test):0.

In [ ]:
df_10InNe_2HidLa_music.head()
# df_10InNe_2HidLa_music.loc[df_10InNe_2HidLa_music['test size'] == 0.8]
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None
                       ):
  display(df_10InNe_2HidLa_music.sort_values(by=['MSE(test)']))
df_10InNe_2HidLa_music.to_csv(os.path.join(result_csv_dir, 'df_10InNe_2HidLa_music.csv'), index=False)

,#neurons por layer oculto,#layers ocultos,max epochs,test size,lrate,alpha,MSE(training),MSE(test)
0,10,2,200,0.5,0.1,0.5,0.054973,0.075773
0,10,2,200,0.5,0.1,0.7,0.053859,0.075833
0,10,2,200,0.5,0.1,0.2,0.057155,0.075877
0,10,2,200,0.5,0.1,1,0.052494,0.076006
0,10,2,200,0.5,0.1,0,0.059062,0.076220
0,10,2,100,0.5,0.9,0,0.048429,0.076477
0,10,2,100,0.5,0.5,0,0.054410,0.076615
0,10,2,100,0.5,0.1,1,0.061081,0.076652
0,10,2,100,0.5,0.9,0.2,0.046768,0.076670
0,10,2,100,0.5,0.5,0.2,0.052715,0.077065


In [ ]:
df_100InNe_2HidLa_music = test_hiperparameters(X_data = music_X, y_data = music_y, classification=False, n_intermed_layers = 2, neur_intermed = 100)

<ipython-input-3-b5beb518ad31>:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([pd.DataFrame([[neur_intermed, n_intermed_layers, max_epochs, test_size, lrate_i, alpha_i, error[-1], MSE]], columns=df.columns), df])


max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.00	  MSE(training):0.05854	  MSE(test):0.09078
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.00	  MSE(training):0.04600	  MSE(test):0.08889
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.00	  MSE(training):0.02949	  MSE(test):0.08922
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.20	  MSE(training):0.05857	  MSE(test):0.09046
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.20	  MSE(training):0.04458	  MSE(test):0.08973
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.20	  MSE(training):0.02688	  MSE(test):0.09334
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.50	  MSE(training):0.05856	  MSE(test):0.09056
max_epochs:100	 test_size:0.20	  lrate:0.10	  alpha:0.50	  MSE(training):0.04279	  MSE(test):0.08764
max_epochs:200	 test_size:0.20	  lrate:0.10	  alpha:0.50	  MSE(training):0.02394	  MSE(test):0.09853
max_epochs:50	 test_size:0.20	  lrate:0.10	  alpha:0.70	  MSE(training):0.05818	  MSE(test):0.

In [ ]:
df_100InNe_2HidLa_music.head()
# df_100InNe_2HidLa_music.loc[df_100InNe_2HidLa_music['test size'] == 0.8]
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None
                       ):
  display(df_100InNe_2HidLa_music.sort_values(by=['MSE(test)']))
df_100InNe_2HidLa_music.to_csv(os.path.join(result_csv_dir, 'df_100InNe_2HidLa_music.csv'), index=False)

,#neurons por layer oculto,#layers ocultos,max epochs,test size,lrate,alpha,MSE(training),MSE(test)
0,100,2,50,0.5,0.1,0,0.057289,0.082024
0,100,2,50,0.5,0.1,0.2,0.055656,0.082463
0,100,2,100,0.5,0.1,0,0.044563,0.083079
0,100,2,100,0.5,0.1,0.2,0.042031,0.083168
0,100,2,100,0.2,0.1,1,0.038212,0.083495
0,100,2,200,0.5,0.1,0,0.028916,0.085461
0,100,2,50,0.5,0.1,0.5,0.054779,0.085538
0,100,2,100,0.5,0.1,0.5,0.040063,0.085726
0,100,2,200,0.5,0.1,0.2,0.026045,0.086031
0,100,2,50,0.5,0.1,0.7,0.054400,0.087335
